# No, Python is Not Too Slow for Computational Economics

### John Stachurski

#### Australian National University

Acknowledgements: Thanks to anonymous friends for helpful comments

Date: September 2018

## Introduction

In [this paper](https://github.com/jstac/julia_python_comparison/blob/master/Update_March_23_2018.pdf), S. Boragan Aruoba and Jesus Fernandez-Villaverde (AFV) study the relative speed of a range of programming lanugages by testing them on a value function iteration routine.  In the abstract they state:

**"The central conclusions of our original paper remain unaltered: C++ is the fastest alternative, Julia offers a great balance of speed and ease of use, and Python is too slow."**

Even the authors' own findings do not support this conclusion.  In their table 1, Python combined with its scientific libraries is actually reported as slightly faster than Julia (2.31 seconds for Python vs 2.35 for Julia).

We rerun the code below and find a similar outcome: The Python code runs in around 3.5 seconds, compared to Julia's 4.5.  The execution environment is 

```
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               142
Model name:          Intel(R) Core(TM) i5-7300U CPU @ 2.60GHz
```

We are using the latest version of each language and scientific ecosystem at the time of writing: Julia 1.0 and Anaconda Python 5.2.

Of course, we are comparing only Julia vs Python plus its scientific libraries (as opposed to Julia vs pure Python), since no scientist would consider foregoing the scientific libraries when implementing computationally intensive algorithms in Python.



## Additional Comments

1. Python is massively popular for both general purpose and scientific computing (see, e.g., [this ranking](https://spectrum.ieee.org/static/interactive-the-top-programming-languages-2018)).  Its popularity is partly due to (and partly caused by) the huge range of third party libraries.  Within the scientific domain, these libraries include high quality array processing software and a powerful JIT compiler.  

2. Python's primary scientific JIT compiler (Numba) and Julia's JIT compiler are built on the same architecture (LLVM).  Similar execution times are to be expected.  From this perspective it is surprising that the Python code runs faster on this machine.  Other users are likely to get slightly different relative run times.

3. AFV do not display any profiling of their codes in the different languages. No memory footprint, no flop rates, etc. are shown. This matters because memory access patterns can always be optimized, especially in small problems as treated here.  One would have to take into account all the individual features of the programming languages to make a fair comparison. (Most likely there would be no surprise, as their performance behavior is known.)  A useful refernence is: S. Goedecker and A. Hoisie. Performance Optimization of Numerically Intensive Codes (SIAM), 2001. ISBN 978-0898714845. 

4. As an aside, the author thinks Julia is a great project and hopes that it succeeds.  Python wouldn't be where it is now without the flow of ideas (and competitive pressure) from Julia.


## Timings

The code below is exactly as in the [GitHub](https://github.com/jesusfv/Comparison-Programming-Languages-Economics) repo, except for the following changes, consistent comments made in the 2018 paper. 

1. Doubled the length of the capital grid.
2. Convergence tolerance is 1e-8.
3. Print out status after every 20 iterations.
4. Changed print statements to print functions (Python 3 compliance)

In [6]:
# Basic RBC model with full depreciation (Alternate 1)
#
# Jesus Fernandez-Villaverde
# Haverford, July 3, 2013

import numpy as np
import math
import time
from numba import jit

# - Start Inner Loop - #
# - bbeta                   float
# - nGridCapital:           int64
# - gridCapitalNextPeriod:  int64
# - mOutput:                float (17820 x 5)
# - nProductivity:          int64
# - vGridCapital:           float (17820, )
# - mValueFunction:         float (17820 x 5)
# - mPolicyFunction:        float (17820 x 5)

@jit
def innerloop(bbeta, nGridCapital, gridCapitalNextPeriod, mOutput, nProductivity, vGridCapital, expectedValueFunction, mValueFunction, mValueFunctionNew, mPolicyFunction):

    for nCapital in range(nGridCapital):
        valueHighSoFar = -100000.0
        capitalChoice  = vGridCapital[0]
        
        for nCapitalNextPeriod in range(gridCapitalNextPeriod, nGridCapital):
            consumption = mOutput[nCapital,nProductivity] - vGridCapital[nCapitalNextPeriod]
            valueProvisional = (1-bbeta)*np.log(consumption)+bbeta*expectedValueFunction[nCapitalNextPeriod,nProductivity];

            if  valueProvisional > valueHighSoFar:
                valueHighSoFar = valueProvisional
                capitalChoice = vGridCapital[nCapitalNextPeriod]
                gridCapitalNextPeriod = nCapitalNextPeriod
            else:
                break 

        mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
        mPolicyFunction[nCapital,nProductivity]   = capitalChoice

    return mValueFunctionNew, mPolicyFunction

def main_func():

    #  1. Calibration

    aalpha = 1.0/3.0     # Elasticity of output w.r.t. capital
    bbeta  = 0.95        # Discount factor

    # Productivity values
    vProductivity = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212],float)

    # Transition matrix
    mTransition   = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
                     [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
                     [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
                     [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
                     [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]],float)

    ## 2. Steady State

    capitalSteadyState     = (aalpha*bbeta)**(1/(1-aalpha))
    outputSteadyState      = capitalSteadyState**aalpha
    consumptionSteadyState = outputSteadyState-capitalSteadyState

    print("Output = ", outputSteadyState, " Capital = ", capitalSteadyState, " Consumption = ", consumptionSteadyState) 

    # We generate the grid of capital
    vGridCapital           = np.arange(0.5*capitalSteadyState,1.5*capitalSteadyState,0.000005)

    nGridCapital           = len(vGridCapital)
    nGridProductivity      = len(vProductivity)

    ## 3. Required matrices and vectors

    mOutput           = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunction    = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunctionNew = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mPolicyFunction   = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    expectedValueFunction = np.zeros((nGridCapital,nGridProductivity),dtype=float)

    # 4. We pre-build output for each point in the grid

    for nProductivity in range(nGridProductivity):
        mOutput[:,nProductivity] = vProductivity[nProductivity]*(vGridCapital**aalpha)

    ## 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.00000001
    iteration = 0

    log = math.log
    zeros = np.zeros
    dot = np.dot

    while(maxDifference > tolerance):

        expectedValueFunction = dot(mValueFunction,mTransition.T)

        for nProductivity in range(nGridProductivity):

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 0

            # - Start Inner Loop - #

            mValueFunctionNew, mPolicyFunction = innerloop(bbeta, nGridCapital, gridCapitalNextPeriod, mOutput, nProductivity, vGridCapital, expectedValueFunction, mValueFunction, mValueFunctionNew, mPolicyFunction)

            # - End Inner Loop - #

        maxDifference = (abs(mValueFunctionNew-mValueFunction)).max()

        mValueFunction    = mValueFunctionNew
        mValueFunctionNew = zeros((nGridCapital,nGridProductivity),dtype=float)

        iteration += 1
        if(iteration%20 == 0 or iteration == 1):
            print(" Iteration = ", iteration, ", Sup Diff = ", maxDifference)

    return (maxDifference, iteration, mValueFunction, mPolicyFunction)

if __name__ == '__main__':
    # - Start Timer - #
    t1=time.time()
    # - Call Main Function - #
    maxDiff, iterate, mValueF, mPolicyFunction = main_func()
    # - End Timer - #
    t2 = time.time()
    print(" Iteration = ", iterate, ", Sup Duff = ", maxDiff)
    print(" ")
    print(" My Check = ", mPolicyFunction[1000-1,3-1])
    print(" ")
    print("Elapse time = is ", t2-t1)

Output =  0.5627314338711378  Capital =  0.178198287392527  Consumption =  0.3845331464786108
 Iteration =  1 , Sup Diff =  0.05274159340733661
 Iteration =  20 , Sup Diff =  0.018703459886607154
 Iteration =  40 , Sup Diff =  0.006668541708074516
 Iteration =  60 , Sup Diff =  0.0023813118038720216
 Iteration =  80 , Sup Diff =  0.0008513397746897633
 Iteration =  100 , Sup Diff =  0.00030462324420166276
 Iteration =  120 , Sup Diff =  0.00010906950872124899
 Iteration =  140 , Sup Diff =  3.907108211809174e-05
 Iteration =  160 , Sup Diff =  1.4008644636520629e-05
 Iteration =  180 , Sup Diff =  5.026474537817016e-06
 Iteration =  200 , Sup Diff =  1.8035522479920019e-06
 Iteration =  220 , Sup Diff =  6.471316944534067e-07
 Iteration =  240 , Sup Diff =  2.3219657940831695e-07
 Iteration =  260 , Sup Diff =  8.331409406814316e-08
 Iteration =  280 , Sup Diff =  2.989378011797328e-08
 Iteration =  300 , Sup Diff =  1.0726129207050406e-08
 Iteration =  302 , Sup Duff =  9.681200952016

Another claim of the paper is that "Julia benefits a lot from a mild investment on optimization and makes it easier to compare with `Numba` and `Cython`, which require extra work as well with respect to basic Python". The evidence they provide actually supports the opposite claim. After adapting their original code to Python 3, it turns out that only two lines need to be added (and a few of their inconsistencies deleted) to get `Numba` to run: `from numba import jit` and `@njit`. Not only is this extremely easy, but it is also much less work than AVF put into trying to optimize their Julia code, which can be partially observed by looking at the history of the file [here](https://github.com/jesusfv/Comparison-Programming-Languages-Economics/commits/master/RBC_Julia.jl). For timing purposes, we also remove the print statements, and add a warm-up period. We run the new code in the cell below.

In [5]:
# 0. Initialization
import numpy as np
import time
from numba import njit


@njit
def main_func():

    #  1. Calibration

    aalpha = 1.0/3.0     # Elasticity of output w.r.t. capital
    bbeta  = 0.95        # Discount factor

    # Productivity values
    vProductivity = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212])

    # Transition matrix
    mTransition   = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
                                         [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
                                         [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
                                         [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
                                         [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]])

    ## 2. Steady State

    capitalSteadyState     = (aalpha*bbeta)**(1/(1-aalpha))
    outputSteadyState      = capitalSteadyState**aalpha
    consumptionSteadyState = outputSteadyState-capitalSteadyState

    # We generate the grid of capital
    vGridCapital           = np.arange(0.5*capitalSteadyState,1.5*capitalSteadyState,0.000005)

    nGridCapital           = len(vGridCapital)
    nGridProductivity      = len(vProductivity)

    ## 3. Required matrices and vectors

    mOutput           = np.zeros((nGridCapital,nGridProductivity))
    mValueFunction    = np.zeros((nGridCapital,nGridProductivity))
    mValueFunctionNew = np.zeros((nGridCapital,nGridProductivity))
    mPolicyFunction   = np.zeros((nGridCapital,nGridProductivity))
    expectedValueFunction = np.zeros((nGridCapital,nGridProductivity))

    # 4. We pre-build output for each point in the grid

    for nProductivity in range(nGridProductivity):
        mOutput[:,nProductivity] = vProductivity[nProductivity]*(vGridCapital**aalpha)

    ## 5. Main iteration

    maxDifference = 10.0
    tolerance = 1e-08
    iteration = 0

    log = np.log
    zeros = np.zeros
    dot = np.dot

    while(maxDifference > tolerance):

        expectedValueFunction = dot(mValueFunction,mTransition.T)

        for nProductivity in range(nGridProductivity):

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 0

            for nCapital in range(nGridCapital):

                valueHighSoFar = -100000.0
                capitalChoice  = vGridCapital[0]

                for nCapitalNextPeriod in range(gridCapitalNextPeriod,nGridCapital):

                    consumption = mOutput[nCapital,nProductivity] - vGridCapital[nCapitalNextPeriod]

                    #expectedValueFunction = np.dot(mTransition[nProductivity,:],mValueFunction[nCapitalNextPeriod,:])
                    valueProvisional = (1-bbeta)*log(consumption)+bbeta*expectedValueFunction[nCapitalNextPeriod,nProductivity];

                    if  valueProvisional>valueHighSoFar:
                        valueHighSoFar = valueProvisional
                        capitalChoice = vGridCapital[nCapitalNextPeriod]
                        gridCapitalNextPeriod = nCapitalNextPeriod
                    else:
                        break # We break when we have achieved the max


                mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
                mPolicyFunction[nCapital,nProductivity]   = capitalChoice

        maxDifference = np.amax(np.abs(mValueFunctionNew-mValueFunction))

        mValueFunction    = mValueFunctionNew
        mValueFunctionNew = zeros((nGridCapital,nGridProductivity))

        iteration += 1

    return (maxDifference, iteration, mValueFunction, mPolicyFunction)

t1=time.time()

maxDiff, iter, mValueF, mPolicyFunction = main_func()

t2 = time.time()

print( " Iteration = ", iter, ", Sup Duff = ", maxDiff)
print(" ")
print(" My Check = ", mPolicyFunction[1000-1,3-1] )
print(" ")

print("Elapse time (warm-up) is ", t2-t1)

print(" ")

print("Elapse time (run) is ")
%timeit main_func()

 Iteration =  302 , Sup Duff =  9.681200952016411e-09
 
 My Check =  0.14404414369631843
 
Elapse time (warm-up) is  2.362729072570801
Elapse time (run) is 
1.89 s ± 91.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Julia version

To run the Julia code you will need to download it from [here](https://github.com/jstac/julia_python_comparison) and place it in the present working directory. We remove the print statements and add a warm-up iteration to be consistent with the Python code.

In [4]:
!julia RBC_Julia.jl

First run (warm-up): 
  3.634823 seconds (2.62 M allocations: 1.337 GiB, 4.83% gc time)
Second run: 
  2.584744 seconds (1.84 k allocations: 1.211 GiB, 3.82% gc time)


We are not the only ones who made similar observations. Other researchers report that Python is faster than Julia [here](https://discourse.julialang.org/t/a-comparison-of-programming-languages-in-economics/8966/19).